In [7]:
import cv2
import mediapipe as mp
import os
import numpy as np

face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

mp_face_detection = mp.solutions.face_detection
mp_drawing = mp.solutions.drawing_utils

image_folder = 'test images'

haar_output_folder = 'haar'
mp_output_folder = 'mediapipe'

for filename in os.listdir(image_folder):
    if filename.endswith(('.png', '.jpg', '.jpeg')):
        image_path = os.path.join(image_folder, filename)
        original_image = cv2.imread(image_path)

        #HaarCascade
        gray = cv2.cvtColor(original_image, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.1, 4)
        haar_image = original_image.copy()
        for (x, y, w, h) in faces:
            cv2.rectangle(haar_image, (x, y), (x+w, y+h), (255, 0, 0), 5)
        cv2.imwrite(os.path.join(haar_output_folder, filename), haar_image)

        #MediaPipe
        mp_image = original_image.copy()
        with mp_face_detection.FaceDetection(min_detection_confidence=0.3) as face_detection:
            results = face_detection.process(cv2.cvtColor(mp_image, cv2.COLOR_BGR2RGB))
            if results.detections:
                for detection in results.detections:
                    bboxC = detection.location_data.relative_bounding_box
                    ih, iw, _ = mp_image.shape
                    x, y, w, h = int(bboxC.xmin * iw), int(bboxC.ymin * ih), \
                                 int(bboxC.width * iw), int(bboxC.height * ih)
                    cv2.rectangle(mp_image, (x, y), (x+w, y+h), (0, 255, 0), 5)  
            cv2.imwrite(os.path.join(mp_output_folder, filename), mp_image)


In [4]:
import cv2

cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("Cannot open camera")
    exit()

while True:
    ret, frame = cap.read()

    if not ret:
        break

    cv2.imshow('Webcam', frame)

    if cv2.waitKey(1) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()